# Capstone 2: Data Wrangling

## Imports

In [58]:
# import 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

## Managers Data

### Column Descriptions
index - row number

owner - manager

draft_vorp - total value over replacement of players gathered during the draft

gm_vorp - total value over replacement of players added off waivers or traded for after the draft

mvp - most valuable player by total value over replacement for the selected manager in the given year

mvp_points - total fantasy points scored by the mvp

mvp_vorp - total value over replacement for the mvp

coaching_points - total points scored by the team minus score team would have scored by following ESPNs projected best 
lineup each week

draft_rank - rank of draft vorp relative to rest of the league

gm_rank - rank of gm_vorp relative to rest of the league

coach_rank - rank of coaching_points relative to rest of the league

owner_value - weighted sum of draft vorp

owner_rank - overall ranking of the owner relative to the rest of the league

year - season year

In [2]:
# import data
data = pd.read_csv("data/managers.csv")

# transform to data frame
managers = pd.DataFrame(data)

# show first 5 rows of data
managers.head()

,Unnamed: 0,owner,draft_vorp,gm_vorp,mvp,mvp_points,mvp_vorp,coaching_points,draft_rank,gm_rank,coach_rank,owner_value,owner_rank,lid,year
0,0,Andrew Nadeau,-536.44,-563.78,Saquon Barkley,198.90,76.40,218.70,1.0,10.0,8.0,-881.52,8.0,64582505,2019
1,1,Emeric Rochford,-1080.96,-59.96,Aaron Jones,247.60,116.35,254.00,10.0,1.0,5.0,-886.92,9.0,64582505,2019
2,2,Fox Winters,-833.06,-249.24,Julio Jones,213.30,88.75,283.00,9.0,5.0,4.0,-799.30,6.0,64582505,2019
3,3,Hatcher D'Agostino,-770.30,-236.68,Patrick Mahomes,275.98,111.55,289.50,5.0,4.0,3.0,-717.48,2.0,64582505,2019
4,4,Jake Landry,-770.68,-197.66,Ezekiel Elliott,258.80,125.20,241.56,6.0,2.0,7.0,-726.78,3.0,64582505,2019


In [3]:
# inspect columns
managers.columns

Index(['Unnamed: 0', 'owner', 'draft_vorp', 'gm_vorp', 'mvp', 'mvp_points',
       'mvp_vorp', 'coaching_points', 'draft_rank', 'gm_rank', 'coach_rank',
       'owner_value', 'owner_rank', 'lid', 'year'],
      dtype='object')

In [4]:
# Drop columns
managers = managers.drop(['Unnamed: 0', 'lid'], axis = 1)
managers.columns

Index(['owner', 'draft_vorp', 'gm_vorp', 'mvp', 'mvp_points', 'mvp_vorp',
       'coaching_points', 'draft_rank', 'gm_rank', 'coach_rank', 'owner_value',
       'owner_rank', 'year'],
      dtype='object')

In [5]:
# Check for missing values
managers.isna().sum()

owner              0
draft_vorp         0
gm_vorp            0
mvp                0
mvp_points         0
mvp_vorp           0
coaching_points    0
draft_rank         0
gm_rank            0
coach_rank         0
owner_value        0
owner_rank         0
year               0
dtype: int64

In [6]:
# Check column types and info
managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   owner            30 non-null     object 
 1   draft_vorp       30 non-null     float64
 2   gm_vorp          30 non-null     float64
 3   mvp              30 non-null     object 
 4   mvp_points       30 non-null     float64
 5   mvp_vorp         30 non-null     float64
 6   coaching_points  30 non-null     float64
 7   draft_rank       30 non-null     float64
 8   gm_rank          30 non-null     float64
 9   coach_rank       30 non-null     float64
 10  owner_value      30 non-null     float64
 11  owner_rank       30 non-null     float64
 12  year             30 non-null     int64  
dtypes: float64(10), int64(1), object(2)
memory usage: 3.2+ KB


## Weekly Team Data

### Column Descriptions
index - row number

week - week in the season

team_id - team id number

team_name - team name

owner - manager name

real_score - total points scored on the week

projected score - projected total points scored on the week

positions - stating positions

scores - scores at each starting position (aligned with the positions list)

espn_score - total points team would have scored if team had started ESPN's projected best lineup

year - season year

vorps - value over replacement of each started on the week

coaching_points - total points scored minus total points team would have scored with ESPN's projected lineup

In [7]:
# import data 
data = pd.read_csv('data/team_data.csv')

# make data frame
team_data = pd.DataFrame(data)

# display head
team_data.head()

,Unnamed: 0,week,team_id,team_name,owner,real_score,proj_score,positions,scores,espn_score,year,lid,vorps,coaching_points
0,0,11,7,Rick Ross's Sloppy Mudpies,Emeric Rochford,133.04,109.86,"{WR,RB,RB2,TE,D/ST,WR2,QB,K,RB/WR/TE}","{0.0,22.0,8.5,10.1,16.0,26.4,33.84,9.0,7.2}",109.84,2019,64582505,"{-2.2,4.800000000000001,-8.7,5.3,24.2,14.06000...",23.20
1,1,1,6,PAWG Patrol,Hatcher D'Agostino,100.60,106.14,"{QB,WR,RB,TE,WR2,RB/WR/TE,D/ST,K,RB2}","{27.5,13.1,21.6,7.5,8.6,2.9,1.0,12.0,6.4}",96.70,2020,64582505,"{14.5,-2.0999999999999996,15.200000000000001,-...",3.90
2,2,1,7,Rick Ross's Sloppy Mudpies,Emeric Rochford,84.86,113.64,"{WR,WR2,TE,RB,RB2,QB,RB/WR/TE,K,D/ST}","{3.1,10.8,7.9,9.4,22.7,5.36,8.6,16.0,1.0}",75.26,2019,64582505,"{-0.7999999999999998,6.9,-12.1,4.1000000000000...",9.60
3,3,1,8,Yang Gang Chain Gang,Andrew Nadeau,99.66,101.71,"{RB,WR,RB2,QB,TE,D/ST,K,WR2,RB/WR/TE}","{15.9,22.3,10.7,18.56,0.0,1.0,5.0,19.6,6.6}",94.30,2019,64582505,"{10.600000000000001,18.400000000000002,5.39999...",5.36
4,4,1,5,No Turbo Nick,Jonny Perreault,125.00,102.80,"{RB,RB2,TE,WR,RB/WR/TE,D/ST,QB,K,WR2}","{20.4,2.6,8.0,9.7,25.4,10.0,16.6,11.0,21.3}",89.60,2019,64582505,"{15.099999999999998,-2.6999999999999997,-12.0,...",35.40


In [8]:
# inspect columns
team_data.columns

Index(['Unnamed: 0', 'week', 'team_id', 'team_name', 'owner', 'real_score',
       'proj_score', 'positions', 'scores', 'espn_score', 'year', 'lid',
       'vorps', 'coaching_points'],
      dtype='object')

In [9]:
# drop columns
team_data = team_data.drop(['Unnamed: 0', 'lid'], axis = 1)
team_data.columns

Index(['week', 'team_id', 'team_name', 'owner', 'real_score', 'proj_score',
       'positions', 'scores', 'espn_score', 'year', 'vorps',
       'coaching_points'],
      dtype='object')

In [10]:
# rename columns
team_data = team_data.rename(columns={'proj_score':'projected score','espn_score':'ESPN best score'})
team_data = team_data.rename(columns={'team_name':'team'})
team_data.columns

Index(['week', 'team_id', 'team', 'owner', 'real_score', 'projected score',
       'positions', 'scores', 'ESPN best score', 'year', 'vorps',
       'coaching_points'],
      dtype='object')

In [11]:
# round column values
team_data = team_data.round({'vorps':2})
team_data.head()

,week,team_id,team,owner,real_score,projected score,positions,scores,ESPN best score,year,vorps,coaching_points
0,11,7,Rick Ross's Sloppy Mudpies,Emeric Rochford,133.04,109.86,"{WR,RB,RB2,TE,D/ST,WR2,QB,K,RB/WR/TE}","{0.0,22.0,8.5,10.1,16.0,26.4,33.84,9.0,7.2}",109.84,2019,"{-2.2,4.800000000000001,-8.7,5.3,24.2,14.06000...",23.20
1,1,6,PAWG Patrol,Hatcher D'Agostino,100.60,106.14,"{QB,WR,RB,TE,WR2,RB/WR/TE,D/ST,K,RB2}","{27.5,13.1,21.6,7.5,8.6,2.9,1.0,12.0,6.4}",96.70,2020,"{14.5,-2.0999999999999996,15.200000000000001,-...",3.90
2,1,7,Rick Ross's Sloppy Mudpies,Emeric Rochford,84.86,113.64,"{WR,WR2,TE,RB,RB2,QB,RB/WR/TE,K,D/ST}","{3.1,10.8,7.9,9.4,22.7,5.36,8.6,16.0,1.0}",75.26,2019,"{-0.7999999999999998,6.9,-12.1,4.1000000000000...",9.60
3,1,8,Yang Gang Chain Gang,Andrew Nadeau,99.66,101.71,"{RB,WR,RB2,QB,TE,D/ST,K,WR2,RB/WR/TE}","{15.9,22.3,10.7,18.56,0.0,1.0,5.0,19.6,6.6}",94.30,2019,"{10.600000000000001,18.400000000000002,5.39999...",5.36
4,1,5,No Turbo Nick,Jonny Perreault,125.00,102.80,"{RB,RB2,TE,WR,RB/WR/TE,D/ST,QB,K,WR2}","{20.4,2.6,8.0,9.7,25.4,10.0,16.6,11.0,21.3}",89.60,2019,"{15.099999999999998,-2.6999999999999997,-12.0,...",35.40


In [12]:
# Check for missing values
team_data.isna().sum()

week               0
team_id            0
team               0
owner              0
real_score         0
projected score    0
positions          0
scores             0
ESPN best score    0
year               0
vorps              0
coaching_points    0
dtype: int64

In [13]:
# Check column types and info
team_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   week             454 non-null    int64  
 1   team_id          454 non-null    int64  
 2   team             454 non-null    object 
 3   owner            454 non-null    object 
 4   real_score       454 non-null    float64
 5   projected score  454 non-null    float64
 6   positions        454 non-null    object 
 7   scores           454 non-null    object 
 8   ESPN best score  454 non-null    float64
 9   year             454 non-null    int64  
 10  vorps            454 non-null    object 
 11  coaching_points  454 non-null    float64
dtypes: float64(4), int64(3), object(5)
memory usage: 42.7+ KB


## Year End Record Data
### Column Descriptions
index - row number

owner - manager name

team - team name

final standing - final standing in the league after the playoffs

reg standing - regular season standing in the league before the playoffs

team wins - total wins

team losses - total losses

points for - total points scored

points against - total points scored by the opponent each week

logo - url of team logo

league wins - total wins if the team played every team in the league every week

league losses - total losses if the team played every team in the league every week

league ties - total ties if the team played every team in the league every week

team id - team id number

elo - strength calculation of the team based on who they beat each week

year - season year

In [14]:
# import data 
data = pd.read_csv('data/record.csv')

# make data frame
year_end = pd.DataFrame(data)

# display head
year_end.head()

,Unnamed: 0,owner,team,final_standing,reg_standing,team_wins,team_losses,points_for,points_agains,logo,league_wins,league_losses,league_ties,team_id,elo,year,lid
0,0,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.80,1379.36,https://www.losangelesblade.com/content/files/...,103,40,0,1,1507.160868,2019,64582505
1,1,Fox Winters,The Foxy Ladies,5,6,6,7,1382.32,1459.82,https://3.bp.blogspot.com/-0bRXw_5jXz0/XGSmSbW...,75,67,0,2,1332.082231,2019,64582505
2,2,nick brustin,Pitter Patter,8,8,5,8,1324.10,1537.12,https://images-na.ssl-images-amazon.com/images...,58,84,0,3,1386.693516,2019,64582505
3,3,James Gaudreault,Bonerville Cougers,3,2,9,4,1387.02,1329.86,https://pbs.twimg.com/profile_images/104511572...,78,64,1,4,1500.030128,2019,64582505
4,4,Jonny Perreault,No Turbo Nick,9,7,6,7,1357.06,1430.82,http://www.enjoytechnica.com/wp-content/upload...,62,79,1,5,1355.372818,2019,64582505


In [15]:
# inspect columns
year_end.columns

Index(['Unnamed: 0', 'owner', 'team', 'final_standing', 'reg_standing',
       'team_wins', 'team_losses', 'points_for', 'points_agains', 'logo',
       'league_wins', 'league_losses', 'league_ties', 'team_id', 'elo', 'year',
       'lid'],
      dtype='object')

In [16]:
# drop columns
year_end = year_end.drop(['Unnamed: 0', 'lid','logo'], axis = 1)
year_end.columns

Index(['owner', 'team', 'final_standing', 'reg_standing', 'team_wins',
       'team_losses', 'points_for', 'points_agains', 'league_wins',
       'league_losses', 'league_ties', 'team_id', 'elo', 'year'],
      dtype='object')

In [17]:
# Check for missing values
year_end.isna().sum()

owner             0
team              0
final_standing    0
reg_standing      0
team_wins         0
team_losses       0
points_for        0
points_agains     0
league_wins       0
league_losses     0
league_ties       0
team_id           0
elo               0
year              0
dtype: int64

In [18]:
# check column types and info
year_end.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   owner           30 non-null     object 
 1   team            30 non-null     object 
 2   final_standing  30 non-null     int64  
 3   reg_standing    30 non-null     int64  
 4   team_wins       30 non-null     int64  
 5   team_losses     30 non-null     int64  
 6   points_for      30 non-null     float64
 7   points_agains   30 non-null     float64
 8   league_wins     30 non-null     int64  
 9   league_losses   30 non-null     int64  
 10  league_ties     30 non-null     int64  
 11  team_id         30 non-null     int64  
 12  elo             30 non-null     float64
 13  year            30 non-null     int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 3.4+ KB


## Weekly Scores
### Column Descriptions
index - row number

week - week in the season

team id - team id number

real score - total points scored

opp score - total points scored by the opponent

owner - team manager name

team - team name

year - season year

In [19]:
# import data 
data = pd.read_csv('data/weekly_scores.csv')

# make data frame
weekly_scores = pd.DataFrame(data)

# display head
weekly_scores.head()

,Unnamed: 0,week,team_id,real_score,opp_score,owner,team,year,lid
0,0,1.0,7,84.86,99.66,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,64582505
1,1,2.0,7,82.32,88.66,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,64582505
2,2,3.0,7,124.98,138.10,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,64582505
3,3,4.0,7,76.50,112.22,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,64582505
4,4,5.0,7,119.94,106.44,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,64582505


In [20]:
# inspect columns
weekly_scores.columns

Index(['Unnamed: 0', 'week', 'team_id', 'real_score', 'opp_score', 'owner',
       'team', 'year', 'lid'],
      dtype='object')

In [21]:
# drop columns
weekly_scores = weekly_scores.drop(['Unnamed: 0', 'lid'], axis = 1)
weekly_scores.columns

Index(['week', 'team_id', 'real_score', 'opp_score', 'owner', 'team', 'year'], dtype='object')

In [22]:
# Check for missing values
weekly_scores.isna().sum()

week          0
team_id       0
real_score    0
opp_score     0
owner         0
team          0
year          0
dtype: int64

In [23]:
# check column types and info
weekly_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   week        454 non-null    float64
 1   team_id     454 non-null    int64  
 2   real_score  454 non-null    float64
 3   opp_score   454 non-null    float64
 4   owner       454 non-null    object 
 5   team        454 non-null    object 
 6   year        454 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 25.0+ KB


## Head To Head Data
### Column Descriptions
index - row number

owner 1 - first manager to compare

owner 2 - second manager to compare

wins - how many games manager 1 has won vs manager 2

losses - how many games manager 2 has won ve manager 1

ties - how many games manager 1 and 2 have tied

point_dif - overall point differential across all games between manager 1 and 2. positive favors manager 1

o1_scores - manager 1 scores in each matchup between manager 1 and 2

o2_scores - manager 2 scores in each matchup between manager 1 and 2

In [24]:
# import data 
data = pd.read_csv('data/h2h.csv')

# make data frame
h2h = pd.DataFrame(data)

# display head
h2h.head()

,Unnamed: 0,owner1,owner2,wins,losses,ties,point_dif,o1_scores,o2_scores,lid,num_matchups
0,0,Andrew Nadeau,Jake Landry,3,5,0,-86.14,"{100.8,45.92,66.96,117.34,89.2,115.22,136.48,8...","{70.86,115.4,108.6,94.36,91.94,119.06,117.78,1...",64582505,8
1,1,Jake Landry,Andrew Nadeau,5,3,0,86.14,"{70.86,115.4,108.6,94.36,91.94,119.06,117.78,1...","{100.8,45.92,66.96,117.34,89.2,115.22,136.48,8...",64582505,8
2,2,James Gaudreault,Hatcher D'Agostino,3,5,0,-123.26,"{115.82,104.62,112.42,110.94,95.0,66.66,94.54,...","{150.24,109.54,142.7,85.1,79.48,126.52,130.72,...",64582505,8
3,3,Hatcher D'Agostino,James Gaudreault,5,3,0,123.26,"{150.24,109.54,142.7,85.1,79.48,126.52,130.72,...","{115.82,104.62,112.42,110.94,95.0,66.66,94.54,...",64582505,8
4,4,Emeric Rochford,Jonny Perreault,4,3,0,108.44,"{124.98,90.4,133.7,147.78,130.9,62.82,115.6}","{138.1,91.8,118.9,72.24,77.58,99.5,99.62}",64582505,7


In [25]:
# inspect columns
h2h.columns

Index(['Unnamed: 0', 'owner1', 'owner2', 'wins', 'losses', 'ties', 'point_dif',
       'o1_scores', 'o2_scores', 'lid', 'num_matchups'],
      dtype='object')

In [26]:
# drop columns
h2h = h2h.drop(['Unnamed: 0', 'lid'], axis = 1)
h2h.columns

Index(['owner1', 'owner2', 'wins', 'losses', 'ties', 'point_dif', 'o1_scores',
       'o2_scores', 'num_matchups'],
      dtype='object')

In [27]:
# Check for missing values
h2h.isna().sum()

owner1          0
owner2          0
wins            0
losses          0
ties            0
point_dif       0
o1_scores       0
o2_scores       0
num_matchups    0
dtype: int64

In [28]:
# check column types and info
h2h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   owner1        90 non-null     object 
 1   owner2        90 non-null     object 
 2   wins          90 non-null     int64  
 3   losses        90 non-null     int64  
 4   ties          90 non-null     int64  
 5   point_dif     90 non-null     float64
 6   o1_scores     90 non-null     object 
 7   o2_scores     90 non-null     object 
 8   num_matchups  90 non-null     int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 6.5+ KB


## Merge Data

### First merge: team_data and year_end

In [29]:
# merge team_data and year_end
merged_df = pd.merge(year_end,team_data, on = ['team','year'])
merged_df.columns

Index(['owner_x', 'team', 'final_standing', 'reg_standing', 'team_wins',
       'team_losses', 'points_for', 'points_agains', 'league_wins',
       'league_losses', 'league_ties', 'team_id_x', 'elo', 'year', 'week',
       'team_id_y', 'owner_y', 'real_score', 'projected score', 'positions',
       'scores', 'ESPN best score', 'vorps', 'coaching_points'],
      dtype='object')

In [30]:
# check out merged_df
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 454 entries, 0 to 453
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   owner_x          454 non-null    object 
 1   team             454 non-null    object 
 2   final_standing   454 non-null    int64  
 3   reg_standing     454 non-null    int64  
 4   team_wins        454 non-null    int64  
 5   team_losses      454 non-null    int64  
 6   points_for       454 non-null    float64
 7   points_agains    454 non-null    float64
 8   league_wins      454 non-null    int64  
 9   league_losses    454 non-null    int64  
 10  league_ties      454 non-null    int64  
 11  team_id_x        454 non-null    int64  
 12  elo              454 non-null    float64
 13  year             454 non-null    int64  
 14  week             454 non-null    int64  
 15  team_id_y        454 non-null    int64  
 16  owner_y          454 non-null    object 
 17  real_score      

In [31]:
# display head
merged_df.head(10)

,owner_x,team,final_standing,reg_standing,team_wins,team_losses,points_for,points_agains,league_wins,league_losses,...,week,team_id_y,owner_y,real_score,projected score,positions,scores,ESPN best score,vorps,coaching_points
0,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1,1,Matt Plager,174.12,112.47,"{RB,TE,WR,RB/WR/TE,WR2,QB,K,RB2,D/ST}","{37.9,10.3,12.2,28.4,24.7,21.62,6.0,25.0,8.0}",137.72,"{32.6,-9.7,8.299999999999999,20.8,-4.02,1.0,19.7}",36.4
1,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,2,1,Matt Plager,119.40,119.64,"{RB,TE,RB/WR/TE,WR,QB,K,WR2,RB2,D/ST}","{6.3,20.2,16.4,12.3,15.0,7.0,12.6,26.6,3.0}",98.90,"{-1.0,0.8999999999999986,0.0,2.140000000000000...",20.5
2,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,3,1,Matt Plager,127.54,124.06,"{RB,TE,WR,WR2,RB2,D/ST,K,QB,RB/WR/TE}","{26.3,12.4,2.7,5.9,22.3,11.0,8.0,22.54,16.4}",100.14,"{15.5,2.5,-13.3,-10.1,11.5,2.0,0.879999999999999}",27.4
3,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,4,1,Matt Plager,112.22,120.61,"{RB,TE,WR,D/ST,WR2,RB2,K,QB,RB/WR/TE}","{28.9,12.0,21.2,1.0,4.2,16.0,13.0,7.62,8.3}",105.02,"{17.099999999999998,-5.5,-14.000000000000004,-...",7.2
4,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,5,1,Matt Plager,155.52,124.42,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{44.7,9.0,17.2,22.8,2.0,23.22,11.4,25.2,0.0}",136.62,"{29.900000000000002,-1.8000000000000007,14.899...",18.9
5,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,6,1,Matt Plager,107.02,126.65,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{19.7,7.8,14.6,12.4,13.0,18.22,4.9,4.4,12.0}",95.32,"{6.6,2.0,-9.4,-0.6999999999999993,0.0,13.59999...",11.7
6,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,7,1,Matt Plager,109.66,116.61,"{TE,WR,D/ST,WR2,RB,QB,RB/WR/TE,RB2,K}","{7.4,2.6,19.0,11.7,27.4,20.56,6.9,7.1,7.0}",83.56,"{-5.6,-2.4,6.699999999999999,20.29999999999999...",26.1
7,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,8,1,Matt Plager,132.70,121.04,"{RB,TE,WR,WR2,RB2,RB/WR/TE,K,D/ST,QB}","{25.5,14.3,25.3,8.7,25.6,8.7,8.0,5.0,11.6}",119.00,"{17.2,12.0,6.5,-10.100000000000001,17.3,-2.0,-...",13.7
8,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,9,1,Matt Plager,136.78,126.88,"{RB,TE,WR,RB2,K,QB,WR2,RB/WR/TE,D/ST}","{36.1,9.7,21.2,13.6,5.0,21.48,1.1,9.6,19.0}",109.18,"{34.2,4.199999999999999,12.399999999999999,11....",27.6
9,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,10,1,Matt Plager,155.98,121.14,"{RB,TE,WR,RB2,K,QB,WR2,D/ST,RB/WR/TE}","{23.1,17.0,13.2,27.8,6.0,25.88,11.6,20.0,11.4}",123.18,"{11.700000000000001,10.1,6.999999999999999,16....",32.8


In [32]:
# drop redundant columns and rename
merged_df = merged_df.drop(['owner_y','team_id_y'], axis = 1)
merged_df = merged_df.rename(columns={'owner_x':'owner','team_id_x':'team_id'})

merged_df.head(10)

,owner,team,final_standing,reg_standing,team_wins,team_losses,points_for,points_agains,league_wins,league_losses,...,elo,year,week,real_score,projected score,positions,scores,ESPN best score,vorps,coaching_points
0,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,1,174.12,112.47,"{RB,TE,WR,RB/WR/TE,WR2,QB,K,RB2,D/ST}","{37.9,10.3,12.2,28.4,24.7,21.62,6.0,25.0,8.0}",137.72,"{32.6,-9.7,8.299999999999999,20.8,-4.02,1.0,19.7}",36.4
1,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,2,119.40,119.64,"{RB,TE,RB/WR/TE,WR,QB,K,WR2,RB2,D/ST}","{6.3,20.2,16.4,12.3,15.0,7.0,12.6,26.6,3.0}",98.90,"{-1.0,0.8999999999999986,0.0,2.140000000000000...",20.5
2,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,3,127.54,124.06,"{RB,TE,WR,WR2,RB2,D/ST,K,QB,RB/WR/TE}","{26.3,12.4,2.7,5.9,22.3,11.0,8.0,22.54,16.4}",100.14,"{15.5,2.5,-13.3,-10.1,11.5,2.0,0.879999999999999}",27.4
3,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,4,112.22,120.61,"{RB,TE,WR,D/ST,WR2,RB2,K,QB,RB/WR/TE}","{28.9,12.0,21.2,1.0,4.2,16.0,13.0,7.62,8.3}",105.02,"{17.099999999999998,-5.5,-14.000000000000004,-...",7.2
4,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,5,155.52,124.42,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{44.7,9.0,17.2,22.8,2.0,23.22,11.4,25.2,0.0}",136.62,"{29.900000000000002,-1.8000000000000007,14.899...",18.9
5,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,6,107.02,126.65,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{19.7,7.8,14.6,12.4,13.0,18.22,4.9,4.4,12.0}",95.32,"{6.6,2.0,-9.4,-0.6999999999999993,0.0,13.59999...",11.7
6,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,7,109.66,116.61,"{TE,WR,D/ST,WR2,RB,QB,RB/WR/TE,RB2,K}","{7.4,2.6,19.0,11.7,27.4,20.56,6.9,7.1,7.0}",83.56,"{-5.6,-2.4,6.699999999999999,20.29999999999999...",26.1
7,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,8,132.70,121.04,"{RB,TE,WR,WR2,RB2,RB/WR/TE,K,D/ST,QB}","{25.5,14.3,25.3,8.7,25.6,8.7,8.0,5.0,11.6}",119.00,"{17.2,12.0,6.5,-10.100000000000001,17.3,-2.0,-...",13.7
8,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,9,136.78,126.88,"{RB,TE,WR,RB2,K,QB,WR2,RB/WR/TE,D/ST}","{36.1,9.7,21.2,13.6,5.0,21.48,1.1,9.6,19.0}",109.18,"{34.2,4.199999999999999,12.399999999999999,11....",27.6
9,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1507.160868,2019,10,155.98,121.14,"{RB,TE,WR,RB2,K,QB,WR2,D/ST,RB/WR/TE}","{23.1,17.0,13.2,27.8,6.0,25.88,11.6,20.0,11.4}",123.18,"{11.700000000000001,10.1,6.999999999999999,16....",32.8


In [33]:
# add diff columns
merged_df['diff_in_proj'] = merged_df['real_score'] - merged_df['projected score']
merged_df['ESPN_vs_real'] = merged_df['real_score'] - merged_df['ESPN best score']

# display new columns
merged_df.head(10)

,owner,team,final_standing,reg_standing,team_wins,team_losses,points_for,points_agains,league_wins,league_losses,...,week,real_score,projected score,positions,scores,ESPN best score,vorps,coaching_points,diff_in_proj,ESPN_vs_real
0,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,1,174.12,112.47,"{RB,TE,WR,RB/WR/TE,WR2,QB,K,RB2,D/ST}","{37.9,10.3,12.2,28.4,24.7,21.62,6.0,25.0,8.0}",137.72,"{32.6,-9.7,8.299999999999999,20.8,-4.02,1.0,19.7}",36.4,61.65,36.4
1,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,2,119.40,119.64,"{RB,TE,RB/WR/TE,WR,QB,K,WR2,RB2,D/ST}","{6.3,20.2,16.4,12.3,15.0,7.0,12.6,26.6,3.0}",98.90,"{-1.0,0.8999999999999986,0.0,2.140000000000000...",20.5,-0.24,20.5
2,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,3,127.54,124.06,"{RB,TE,WR,WR2,RB2,D/ST,K,QB,RB/WR/TE}","{26.3,12.4,2.7,5.9,22.3,11.0,8.0,22.54,16.4}",100.14,"{15.5,2.5,-13.3,-10.1,11.5,2.0,0.879999999999999}",27.4,3.48,27.4
3,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,4,112.22,120.61,"{RB,TE,WR,D/ST,WR2,RB2,K,QB,RB/WR/TE}","{28.9,12.0,21.2,1.0,4.2,16.0,13.0,7.62,8.3}",105.02,"{17.099999999999998,-5.5,-14.000000000000004,-...",7.2,-8.39,7.2
4,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,5,155.52,124.42,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{44.7,9.0,17.2,22.8,2.0,23.22,11.4,25.2,0.0}",136.62,"{29.900000000000002,-1.8000000000000007,14.899...",18.9,31.10,18.9
5,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,6,107.02,126.65,"{RB,TE,WR,RB2,K,QB,RB/WR/TE,WR2,D/ST}","{19.7,7.8,14.6,12.4,13.0,18.22,4.9,4.4,12.0}",95.32,"{6.6,2.0,-9.4,-0.6999999999999993,0.0,13.59999...",11.7,-19.63,11.7
6,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,7,109.66,116.61,"{TE,WR,D/ST,WR2,RB,QB,RB/WR/TE,RB2,K}","{7.4,2.6,19.0,11.7,27.4,20.56,6.9,7.1,7.0}",83.56,"{-5.6,-2.4,6.699999999999999,20.29999999999999...",26.1,-6.95,26.1
7,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,8,132.70,121.04,"{RB,TE,WR,WR2,RB2,RB/WR/TE,K,D/ST,QB}","{25.5,14.3,25.3,8.7,25.6,8.7,8.0,5.0,11.6}",119.00,"{17.2,12.0,6.5,-10.100000000000001,17.3,-2.0,-...",13.7,11.66,13.7
8,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,9,136.78,126.88,"{RB,TE,WR,RB2,K,QB,WR2,RB/WR/TE,D/ST}","{36.1,9.7,21.2,13.6,5.0,21.48,1.1,9.6,19.0}",109.18,"{34.2,4.199999999999999,12.399999999999999,11....",27.6,9.90,27.6
9,Matt Plager,Hatcher Top Jonny Bottom,2,1,9,4,1660.8,1379.36,103,40,...,10,155.98,121.14,"{RB,TE,WR,RB2,K,QB,WR2,D/ST,RB/WR/TE}","{23.1,17.0,13.2,27.8,6.0,25.88,11.6,20.0,11.4}",123.18,"{11.700000000000001,10.1,6.999999999999999,16....",32.8,34.84,32.8


#### Create data frame of season points totals
Data frame has total points for, projected points for, and ESPN's best score totals

In [34]:
# do group_by sum
season_points_diff = pd.DataFrame(merged_df.groupby(['owner','year','team',])[['real_score','projected score','ESPN best score']].sum())
season_points_diff.head(25)

real_score  \
owner              year team                                       
Andrew Nadeau      2019 Yang Gang Chain Gang             1368.94   
                   2020 I Don't Tip I Pay Bills          1738.28   
                   2021 TorNadeau's Meadows              1605.08   
Emeric Rochford    2019 Rick Ross's  Sloppy Mudpies      1653.16   
                   2020 Rick Ross's  Sloppy Mudpies      1708.30   
                   2021 Rick Ross's  Sloppy Mudpies      1676.12   
Fox Winters        2019 The Foxy Ladies                  1592.48   
                   2020 The Foxy Ladies                  1781.08   
                   2021 The Foxy Ladies                  1794.96   
Hatcher D'Agostino 2019 PAWG Patrol                      1736.56   
                   2020 PAWG Patrol                      1526.36   
                   2021 PAWG Patrol                      1617.66   
Jake Landry        2019 Riff Raffs Team of All Stars     1467.86   
                   2020 a condum full o precum           1477.20   
                   2021 Tony LaRussa's DUI               1852.16   
James Gaudreault   2019 Bonerville Cougers               1499.44   
                   2020 Bonerville Cougers               1766.90   
                   2021 Bonerville Cougers               1659.28   
Jonny Perreault    2019 No Turbo Nick                    1497.70   
                   2020 Incognito Mode                   1332.34   
                   2021 Big ole doinks  in Amish         1608.62   
Matt  Plager       2019 Hatcher Top Jonny Bottom         1779.20   
                   2020 El Chapo's Guzmen                1566.54   
                   2021 HASBULLA MAGOMEDOV               1886.00   
Mike Kearney       2019 In Crust We Trust                1689.30   

                                                      projected score  \
owner              year team                                            
Andrew Nadeau      2019 Yang Gang Chain Gang                  1506.29   
                   2020 I Don't Tip I Pay Bills               1651.62   
                   2021 TorNadeau's Meadows                   1659.82   
Emeric Rochford    2019 Rick Ross's  Sloppy Mudpies           1647.97   
                   2020 Rick Ross's  Sloppy Mudpies           1614.13   
                   2021 Rick Ross's  Sloppy Mudpies           1774.22   
Fox Winters        2019 The Foxy Ladies                       1670.69   
                   2020 The Foxy Ladies                       1736.27   
                   2021 The Foxy Ladies                       1774.92   
Hatcher D'Agostino 2019 PAWG Patrol                           1707.25   
                   2020 PAWG Patrol                           1615.11   
                   2021 PAWG Patrol                           1595.90   
Jake Landry        2019 Riff Raffs Team of All Stars          1572.65   
                   2020 a condum full o precum                1533.16   
                   2021 Tony LaRussa's DUI                    1848.77   
James Gaudreault   2019 Bonerville Cougers                    1548.39   
                   2020 Bonerville Cougers                    1618.85   
                   2021 Bonerville Cougers                    1750.07   
Jonny Perreault    2019 No Turbo Nick                         1559.66   
                   2020 Incognito Mode                        1446.24   
                   2021 Big ole doinks  in Amish              1608.27   
Matt  Plager       2019 Hatcher Top Jonny Bottom              1700.25   
                   2020 El Chapo's Guzmen                     1691.19   
                   2021 HASBULLA MAGOMEDOV                    1901.52   
Mike Kearney       2019 In Crust We Trust                     1633.36   

                                                      ESPN best score  
owner              year team                                           
Andrew Nadeau      2019 Yang Gang Chain Gang                  1150.24  
                   2020 I Don

In [35]:
# check info
season_points_diff.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 30 entries, ('Andrew Nadeau', 2019, 'Yang Gang Chain Gang') to ('nick brustin', 2021, 'Crustholes Band of Misfits')
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   real_score       30 non-null     float64
 1   projected score  30 non-null     float64
 2   ESPN best score  30 non-null     float64
dtypes: float64(3)
memory usage: 1.2+ KB


### Season Totals DataFrame
Create data frame for season totals for each team by merging merged_df and season_points_diff

In [36]:
# merge data frames
season_merge = pd.merge(season_points_diff, year_end, on = ['team','year','owner'])
season_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   team             30 non-null     object 
 1   year             30 non-null     int64  
 2   owner            30 non-null     object 
 3   real_score       30 non-null     float64
 4   projected score  30 non-null     float64
 5   ESPN best score  30 non-null     float64
 6   final_standing   30 non-null     int64  
 7   reg_standing     30 non-null     int64  
 8   team_wins        30 non-null     int64  
 9   team_losses      30 non-null     int64  
 10  points_for       30 non-null     float64
 11  points_agains    30 non-null     float64
 12  league_wins      30 non-null     int64  
 13  league_losses    30 non-null     int64  
 14  league_ties      30 non-null     int64  
 15  team_id          30 non-null     int64  
 16  elo              30 non-null     float64
dtypes: float64(6), int

In [37]:
# Add diff columns
season_merge['real vs proj'] = season_merge['real_score'] - season_merge['projected score']
season_merge['real vs ESPN'] = season_merge['real_score'] - season_merge['ESPN best score']

# check head
season_merge.head(10)

,team,year,owner,real_score,projected score,ESPN best score,final_standing,reg_standing,team_wins,team_losses,points_for,points_agains,league_wins,league_losses,league_ties,team_id,elo,real vs proj,real vs ESPN
0,Yang Gang Chain Gang,2019,Andrew Nadeau,1368.94,1506.29,1150.24,7,9,5,8,1169.22,1304.28,45,97,0,8,1365.071479,-137.35,218.70
1,I Don't Tip I Pay Bills,2020,Andrew Nadeau,1738.28,1651.62,1392.36,2,5,7,6,1448.70,1510.06,82,60,0,8,1448.277454,86.66,345.92
2,TorNadeau's Meadows,2021,Andrew Nadeau,1605.08,1659.82,1345.42,4,2,9,5,1518.90,1466.46,70,81,1,8,1469.497347,-54.74,259.66
3,Rick Ross's Sloppy Mudpies,2019,Emeric Rochford,1653.16,1647.97,1399.16,6,5,6,7,1422.96,1362.46,71,71,0,7,1361.396567,5.19,254.00
4,Rick Ross's Sloppy Mudpies,2020,Emeric Rochford,1708.30,1614.13,1389.50,1,2,11,2,1581.84,1418.74,96,46,1,7,1612.452105,94.17,318.80
5,Rick Ross's Sloppy Mudpies,2021,Emeric Rochford,1676.12,1774.22,1418.32,8,8,6,8,1454.30,1529.52,58,93,0,7,1424.640646,-98.10,257.80
6,The Foxy Ladies,2019,Fox Winters,1592.48,1670.69,1309.48,5,6,6,7,1382.32,1459.82,75,67,0,2,1332.082231,-78.21,283.00
7,The Foxy Ladies,2020,Fox Winters,1781.08,1736.27,1451.42,3,3,7,6,1562.28,1409.06,85,57,0,2,1405.563957,44.81,329.66
8,The Foxy Ladies,2021,Fox Winters,1794.96,1774.92,1468.76,6,4,8,6,1658.06,1592.48,91,59,1,2,1410.830551,20.04,326.20
9,PAWG Patrol,2019,Hatcher D'Agostino,1736.56,1707.25,1447.06,1,3,8,5,1479.34,1422.16,94,48,0,6,1487.750610,29.31,289.50


### Weekly Scores Data Frame
Taking weekly_scores and merging with team_data

In [38]:
week_merge = pd.merge(weekly_scores, team_data, on=['week','owner','year'])
week_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 454 entries, 0 to 453
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   week             454 non-null    float64
 1   team_id_x        454 non-null    int64  
 2   real_score_x     454 non-null    float64
 3   opp_score        454 non-null    float64
 4   owner            454 non-null    object 
 5   team_x           454 non-null    object 
 6   year             454 non-null    int64  
 7   team_id_y        454 non-null    int64  
 8   team_y           454 non-null    object 
 9   real_score_y     454 non-null    float64
 10  projected score  454 non-null    float64
 11  positions        454 non-null    object 
 12  scores           454 non-null    object 
 13  ESPN best score  454 non-null    float64
 14  vorps            454 non-null    object 
 15  coaching_points  454 non-null    float64
dtypes: float64(7), int64(3), object(6)
memory usage: 60.3+ KB


In [39]:
# drop unnecessary columns
week_merge = week_merge.drop(['team_id_y','real_score_x','team_y','coaching_points','positions','scores','vorps'], axis=1)
week_merge.head(10)

,week,team_id_x,opp_score,owner,team_x,year,real_score_y,projected score,ESPN best score
0,1.0,7,99.66,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,84.86,113.64,75.26
1,2.0,7,88.66,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,82.32,104.55,84.12
2,3.0,7,138.10,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,124.98,100.69,94.48
3,4.0,7,112.22,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,76.50,101.53,65.20
4,5.0,7,106.44,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,119.94,97.45,97.14
5,6.0,7,109.30,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,89.00,107.37,78.20
6,7.0,7,93.64,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,110.08,119.86,106.08
7,8.0,7,121.30,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,166.66,116.72,131.76
8,9.0,7,87.74,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,109.80,115.75,84.30
9,10.0,7,104.22,Emeric Rochford,Rick Ross's Sloppy Mudpies,2019,168.34,118.42,110.54


In [40]:
# move columns
week_merge = week_merge[['owner','team_x','team_id_x','year','week','real_score_y','opp_score',
                         'projected score','ESPN best score']]
week_merge.head(10)

,owner,team_x,team_id_x,year,week,real_score_y,opp_score,projected score,ESPN best score
0,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,1.0,84.86,99.66,113.64,75.26
1,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,2.0,82.32,88.66,104.55,84.12
2,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,3.0,124.98,138.10,100.69,94.48
3,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,4.0,76.50,112.22,101.53,65.20
4,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,5.0,119.94,106.44,97.45,97.14
5,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,6.0,89.00,109.30,107.37,78.20
6,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,7.0,110.08,93.64,119.86,106.08
7,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,8.0,166.66,121.30,116.72,131.76
8,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,9.0,109.80,87.74,115.75,84.30
9,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,10.0,168.34,104.22,118.42,110.54


In [41]:
# rename columns 
week_merge = week_merge.rename(columns={'team_x':'team','team_id_x':'team_id','real_score_y':'real_score'})
week_merge.columns

Index(['owner', 'team', 'team_id', 'year', 'week', 'real_score', 'opp_score',
       'projected score', 'ESPN best score'],
      dtype='object')

In [42]:
# add diff columns
week_merge['diff'] = week_merge['real_score'] - week_merge['opp_score']
week_merge['real vs proj'] = week_merge['real_score'] - week_merge['projected score']
week_merge['real vs ESPN'] = week_merge['real_score'] - week_merge['ESPN best score']

In [43]:
# add rank columns
week_merge['week score rank'] = week_merge.groupby(['year','week'])['real_score'].rank(ascending=False)
week_merge['opp score rank'] = week_merge.groupby(['year','week'])['opp_score'].rank(ascending=False)

In [44]:
# add binary iterator columns

# count if a matchup is a win or not
week_merge['is win'] = np.where(week_merge['diff'] > 0, 1, 0)

# Create column for win/loss type
# create conditions
conditions = [(week_merge['diff'] > 0) & (week_merge['week score rank'] <= 5),
              (week_merge['diff'] < 0) & (week_merge['week score rank'] <= 5),
              (week_merge['diff'] > 1) & (week_merge['week score rank'] > 5),
              (week_merge['diff'] < 0) & (week_merge['opp score rank'] > 5)]

# assign values to conditions
values = [1,2,3,4]
# add to column
week_merge['win/loss type'] = np.select(conditions,values)

# Add columns for top 5 loss, bottom 5 loss, bottom 5 win, top 5 win
week_merge['top 5 win'] = np.where(week_merge['win/loss type'] == 1, 1, 0)
week_merge['top 5 loss'] = np.where(week_merge['win/loss type'] == 2, 1, 0)
week_merge['bottom 5 win'] = np.where(week_merge['win/loss type'] == 3, 1, 0)
week_merge['bottom 5 loss'] = np.where(week_merge['win/loss type'] == 4, 1, 0)

# check columns
week_merge.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 454 entries, 0 to 453
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   owner            454 non-null    object 
 1   team             454 non-null    object 
 2   team_id          454 non-null    int64  
 3   year             454 non-null    int64  
 4   week             454 non-null    float64
 5   real_score       454 non-null    float64
 6   opp_score        454 non-null    float64
 7   projected score  454 non-null    float64
 8   ESPN best score  454 non-null    float64
 9   diff             454 non-null    float64
 10  real vs proj     454 non-null    float64
 11  real vs ESPN     454 non-null    float64
 12  week score rank  454 non-null    float64
 13  opp score rank   454 non-null    float64
 14  is win           454 non-null    int64  
 15  win/loss type    454 non-null    int64  
 16  top 5 win        454 non-null    int64  
 17  top 5 loss      

In [45]:
# check head
week_merge.head(10)

,owner,team,team_id,year,week,real_score,opp_score,projected score,ESPN best score,diff,real vs proj,real vs ESPN,week score rank,opp score rank,is win,win/loss type,top 5 win,top 5 loss,bottom 5 win,bottom 5 loss
0,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,1.0,84.86,99.66,113.64,75.26,-14.80,-28.78,9.6,9.0,7.0,0,4,0,0,0,1
1,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,2.0,82.32,88.66,104.55,84.12,-6.34,-22.23,-1.8,8.0,7.0,0,4,0,0,0,1
2,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,3.0,124.98,138.10,100.69,94.48,-13.12,24.29,30.5,7.0,2.0,0,0,0,0,0,0
3,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,4.0,76.50,112.22,101.53,65.20,-35.72,-25.03,11.3,10.0,3.0,0,0,0,0,0,0
4,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,5.0,119.94,106.44,97.45,97.14,13.50,22.49,22.8,5.0,7.0,1,1,1,0,0,0
5,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,6.0,89.00,109.30,107.37,78.20,-20.30,-18.37,10.8,10.0,3.0,0,0,0,0,0,0
6,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,7.0,110.08,93.64,119.86,106.08,16.44,-9.78,4.0,2.0,4.0,1,1,1,0,0,0
7,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,8.0,166.66,121.30,116.72,131.76,45.36,49.94,34.9,1.0,4.0,1,1,1,0,0,0
8,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,9.0,109.80,87.74,115.75,84.30,22.06,-5.95,25.5,5.0,8.0,1,1,1,0,0,0
9,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,10.0,168.34,104.22,118.42,110.54,64.12,49.92,57.8,1.0,5.0,1,1,1,0,0,0


In [46]:
# Sum win/loss type to do a merge with season totals
win_loss_type_df = pd.DataFrame(week_merge.groupby(['owner','year','team',])[['is win','top 5 win','top 5 loss',
                                                                           'bottom 5 win','bottom 5 loss']].sum())
win_loss_type_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 30 entries, ('Andrew Nadeau', 2019, 'Yang Gang Chain Gang') to ('nick brustin', 2021, 'Crustholes Band of Misfits')
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   is win         30 non-null     int64
 1   top 5 win      30 non-null     int64
 2   top 5 loss     30 non-null     int64
 3   bottom 5 win   30 non-null     int64
 4   bottom 5 loss  30 non-null     int64
dtypes: int64(5)
memory usage: 1.7+ KB


In [47]:
# merge season data with win/loss type df
win_loss_season =  pd.merge(season_merge,win_loss_type_df,on=['owner','year','team'])
win_loss_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   team             30 non-null     object 
 1   year             30 non-null     int64  
 2   owner            30 non-null     object 
 3   real_score       30 non-null     float64
 4   projected score  30 non-null     float64
 5   ESPN best score  30 non-null     float64
 6   final_standing   30 non-null     int64  
 7   reg_standing     30 non-null     int64  
 8   team_wins        30 non-null     int64  
 9   team_losses      30 non-null     int64  
 10  points_for       30 non-null     float64
 11  points_agains    30 non-null     float64
 12  league_wins      30 non-null     int64  
 13  league_losses    30 non-null     int64  
 14  league_ties      30 non-null     int64  
 15  team_id          30 non-null     int64  
 16  elo              30 non-null     float64
 17  real vs proj     3

In [48]:
# drop redundant columns
win_loss_season = win_loss_season.drop(['is win'],axis=1)

In [49]:
# set season_merge equal to new df
season_merge = win_loss_season

## Final Table Presentations
### Weekly Totals
#### Columns: 
owner - manager name

team - team name

team id - team id number

year - season year

week - week in the season

real score - total points scored

opp score - total points scored by the opponent

projected score - projected points total for the season

ESPN best score - total score of ESPN's best lineup

diff - points difference between points for and points against

real vs proj - points difference between points for and projected points for

real vs ESPN - pints difference between points for and ESPN's best lineup

week score rank - rank of score for the week compared to every other team's score that week

opp score rank - rank of opponent's score for the week compared to every other team's score that week

is win - shows if the matchup was a win for the team. 1 if win, 0 if a loss

win/loss type - category column to distinguish between win/loss type

top 5 win - counts if the matchup was a top 5 win

top 5 loss - counts if the matchup was a top 5 loss

bottom 5 win - counts if the matchup was a bottom 5 win

bottom 5 loss - counts if the matchup was a bottom 5 loss

In [52]:
# final rename
week_data_clean = week_merge
week_data_clean.head(5)

,owner,team,team_id,year,week,real_score,opp_score,projected score,ESPN best score,diff,real vs proj,real vs ESPN,week score rank,opp score rank,is win,win/loss type,top 5 win,top 5 loss,bottom 5 win,bottom 5 loss
0,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,1.0,84.86,99.66,113.64,75.26,-14.80,-28.78,9.6,9.0,7.0,0,4,0,0,0,1
1,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,2.0,82.32,88.66,104.55,84.12,-6.34,-22.23,-1.8,8.0,7.0,0,4,0,0,0,1
2,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,3.0,124.98,138.10,100.69,94.48,-13.12,24.29,30.5,7.0,2.0,0,0,0,0,0,0
3,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,4.0,76.50,112.22,101.53,65.20,-35.72,-25.03,11.3,10.0,3.0,0,0,0,0,0,0
4,Emeric Rochford,Rick Ross's Sloppy Mudpies,7,2019,5.0,119.94,106.44,97.45,97.14,13.50,22.49,22.8,5.0,7.0,1,1,1,0,0,0


### Season Totals
#### Columns:
team - team name

year - season year

owner - manager name

real score - total points scored 

projected score - projected points total for the season

ESPN best score - total score of ESPN's best lineup

final_standing - final standing for each team during each season after playoffs

reg_standing - final standing for each team during each season before playoffs

team_wins - number of wins for the team on the season

team_losses - number of losses for the team on the season

points_for - total points scored on the season

points_agains - total points scored against the team on the season

league wins - total wins if the team played every team in the league every week

league losses - total losses if the team played every team in the league every week

league ties - total ties if the team played every team in the league every week

team id - team id number

elo - strength calculation of the team based on who they beat each week

real vs proj - points difference between points for and projected points for

real vs ESPN - pints difference between points for and ESPN's best lineup

week score rank - rank of score for the week compared to every other team's score that week

opp score rank - rank of opponent's score for the week compared to every other team's score that week

is win - shows if the matchup was a win for the team. 1 if win, 0 if a loss

win/loss type - category column to distinguish between win/loss type

top 5 win - counts if the matchup was a top 5 win

top 5 loss - counts if the matchup was a top 5 loss

bottom 5 win - counts if the matchup was a bottom 5 win

bottom 5 loss - counts if the matchup was a bottom 5 loss

In [53]:
# final rename
season_data_clean = season_merge
season_data_clean.head(5)

,team,year,owner,real_score,projected score,ESPN best score,final_standing,reg_standing,team_wins,team_losses,...,league_losses,league_ties,team_id,elo,real vs proj,real vs ESPN,top 5 win,top 5 loss,bottom 5 win,bottom 5 loss
0,Yang Gang Chain Gang,2019,Andrew Nadeau,1368.94,1506.29,1150.24,7,9,5,8,...,97,0,8,1365.071479,-137.35,218.70,3,1,3,1
1,I Don't Tip I Pay Bills,2020,Andrew Nadeau,1738.28,1651.62,1392.36,2,5,7,6,...,60,0,8,1448.277454,86.66,345.92,6,3,2,2
2,TorNadeau's Meadows,2021,Andrew Nadeau,1605.08,1659.82,1345.42,4,2,9,5,...,81,1,8,1469.497347,-54.74,259.66,6,0,3,1
3,Rick Ross's Sloppy Mudpies,2019,Emeric Rochford,1653.16,1647.97,1399.16,6,5,6,7,...,71,0,7,1361.396567,5.19,254.00,7,1,0,3
4,Rick Ross's Sloppy Mudpies,2020,Emeric Rochford,1708.30,1614.13,1389.50,1,2,11,2,...,46,1,7,1612.452105,94.17,318.80,10,0,2,0


#### Export to CSVs

In [67]:
# save the data to a new csv file
datapath = Path('data/week_data_clean.csv')
week_data_clean.to_csv(datapath)

In [68]:
# save the data to a new csv file
datapath = Path('data/season_data_clean.csv')
season_data_clean.to_csv(datapath)